In [2]:
#BASE URL AND KEY
#Use the requests library
import csv
import requests as r
import pandas as pd
import zipfile, io
import os

#Set the base URL.
base_url ='https://smmtool.app.cloud.gov/api/' 

# Add a key for the request hhttps://api.edap-cluster.com/useeio/api/eaders. 
# No keys are needed by default, so it's blank

api_headers = {}
# api-key for USEEIO
with open(".BEA_API_KEY", 'r') as KEY_FILE:
    api_headers['x-api-key']=KEY_FILE.read()

In [3]:
import pathlib

# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd().parent
raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'

# Create new folders if needed
state_data_dir = out_data_dir / 'state_level'
if not state_data_dir.exists():
    state_data_dir.mkdir(parents=True)
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)

In [4]:
county_data_dir

PosixPath('/Users/merrelbook/Projects/HealthDataVizGA/DataVisualAnalytics_Industries-cancer/data_raw/BEA_Industry_Factors/county_level')

### Industries' Info per State of GA 
EMP - Total employment  
NESTAB - Number of establishments  
PAYANN - Annual payroll  

To get variables for each year-> https://api.census.gov/data/2011/cbp/variables.html   
General API Documentation-> https://www.census.gov/data/developers/data-sets/cbp-nonemp-zbp/cbp-api.html  
We can go a lot lower hat year 2000 as well!

In [ ]:
years=range(2000,2018)
for year in years:
    print("Getting data for year"+str(year))
    
    # Define the api call --> changes per year based on NAICS coding
    if year>=2000 and year<=2002:
        url="https://api.census.gov/data/"+str(year)+"/cbp?get=GEO_TTL,YEAR,NAICS1997,NAICS1997_TTL,ESTAB,EMP,PAYANN&for=state:13"
    elif year>=2003 and year<=2007:
        url="https://api.census.gov/data/"+str(year)+"/cbp?get=GEO_TTL,YEAR,NAICS2002,NAICS2002_TTL,ESTAB,EMP,PAYANN&for=state:13"
    elif year>=2008 and year<=2011:
        url="https://api.census.gov/data/"+str(year)+"/cbp?get=GEO_TTL,YEAR,NAICS2007,NAICS2007_TTL,ESTAB,EMP,PAYANN&for=state:13"
    elif year>=2012 and year<=2016:
        url="https://api.census.gov/data/"+str(year)+"/cbp?get=GEO_TTL,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN&for=state:13"
    elif year==2017:
        url="https://api.census.gov/data/"+str(year)+"/cbp?get=NAME,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN&for=state:13"
    
    # Query the API
    response = r.get(url)
    
    # Save the API return to CSV
    with open(state_data_dir / f"industriesPerStateGA-{year}.csv",'w') as resultPath:
        for line in response.text.strip().split('\n'):
            line=line.replace('[',"").replace(']',"")
            resultPath.write(line + "\n")

    print("Finished CSV for year"+str(year))

#to read a csv file ----------------------------------------------------------->
#pd1=pd.read_csv("dataSets/industriesPerStateGA"+str(2014)+".csv",delimiter=',')
#pd1

Getting data for year2000
Finished CSV for year2000
Getting data for year2001


### Industries Info per Counties of GA

#### API Comments - County Level

This set of API calls are requesting data on *county business patterns* (CBP)

See https://api.census.gov/data/2017/cbp.html




In [5]:


# years=range(1999,2000)
# count=0
# for year in years:
# #     columns_to_select = "GEO_TTL,COUNTY,YEAR,NAICS1997,NAICS1997_TTL,ESTAB,EMP,PAYANN"
#     columns_to_select = "GEO_TTL,COUNTY,YEAR,ESTAB,EMP,PAYANN,NAICS1997_TTL"

    
# #     url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13&NAICS2007=72"
# #     url="https://api.census.gov/data/"+str(year)+"/cbp?get=GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN&for=county:*&in=state:13"
# #     url = "https://api.census.gov/data/2000/cbp?get=GEO_TTL,NAICS1997_TTL,EMP,ESTAB&for=county:*&in=state:06"
#     url = "https://api.census.gov/data/2010/cbp?get=ESTAB,LFO,NAICS2007,NAICS2007_TTL,GEO_TTL&for=state:06&NAICS2007=72"
#     response = r.get(url)
    
#     print(f'{year} --> API Response: {response.status_code}')

# # response

1999 --> API Response: 200


In [6]:
print(response.text)

[["ESTAB","LFO","NAICS2007","NAICS2007_TTL","GEO_TTL","NAICS2007","state"],
["75463","001","72    ","Accommodation and food services","California","72    ","06"],
["19310","910","72    ","Accommodation and food services","California","72    ","06"],
["22854","9111","72    ","Accommodation and food services","California","72    ","06"],
["18802","920","72    ","Accommodation and food services","California","72    ","06"],
["14090","930","72    ","Accommodation and food services","California","72    ","06"],
["347","932","72    ","Accommodation and food services","California","72    ","06"],
["12","933","72    ","Accommodation and food services","California","72    ","06"],
["48","942","72    ","Accommodation and food services","California","72    ","06"]]


In [7]:
# TO GET AVAILABLE VARIABLES

# year = 2013
# url = f"{base_url}/{year}/cbp/variables"

# response = r.get(url)
# print(response.text)

In [11]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#

years=range(2012,2018)
count=0
for year in years:
    print("Getting data for year"+str(year))
    if year>=2000 and year<=2002:
        columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
        url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13"
    elif year>=2003 and year<=2007:
        columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
        url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13&NAICS2002=70"
    elif year>=2008 and year<=2011:
        columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
        url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13"
    elif year>=2012 and year<=2016:
        columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
        url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13"
    elif year==2017:
        columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
        url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:13"
    
    
    response = r.get(url)
        
    with open(county_data_dir / f"industriesPerCountyGA-{year}.csv",'w') as resultPath:
        for line in response.text.strip().split('\n'):
            line=line.replace('[',"").replace(']',"")
            resultPath.write(line + "\n")

    print("Finished CSV for year"+str(year))

Getting data for year2017
Finished CSV for year2017
